# How to create STAC Catalogs 
## STAC Community Sprint, Arlington, November 7th 2019

This notebook runs through some of the basics of using PySTAC to create a static STAC. It was part of a 30 minute presentation at the [community STAC sprint](https://github.com/radiantearth/community-sprints/tree/master/11052019-arlignton-va) in Arlington, VA in November 2019, updated to work with current PySTAC.

This tutorial will require the `boto3`, `rasterio`, and `shapely` libraries:

In [1]:
%pip install boto3 rasterio shapely pystac

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 132 kB 16.7 MB/s 
     |████████████████████████████████| 19.3 MB 51.1 MB/s 
     |████████████████████████████████| 137 kB 54.5 MB/s 
     |████████████████████████████████| 8.8 MB 54.5 MB/s 
     |████████████████████████████████| 79 kB 10.0 MB/s 
     |████████████████████████████████| 138 kB 39.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


We can import pystac and access most of the functionality we need with the single import:

In [1]:
import pystac

## Creating a catalog from a local file

To give us some material to work with, lets download a single image from the [Spacenet 5 challenge](https://www.topcoder.com/challenges/30099956). We'll use a temporary directory to save off our single-item STAC.

In [12]:
import os
import urllib.request
from tempfile import TemporaryDirectory

tmp_dir = TemporaryDirectory()
img_path = os.path.join(tmp_dir.name, 'image.tif')

In [13]:
# url = ('https://spacenet-dataset.s3.amazonaws.com/'
#        'spacenet/SN5_roads/train/AOI_7_Moscow/MS/'
#        'SN5_roads_train_AOI_7_Moscow_MS_chip996.tif')
# urllib.request.urlretrieve(url, img_path)

We want to create a Catalog. Let's check the pydocs for `Catalog` to see what information we'll need. (We use `__doc__` instead of `help()` here to avoid printing out all the docs for the class.)

In [14]:
print(pystac.Catalog.__doc__)

A PySTAC Catalog represents a STAC catalog in memory.

    A Catalog is a :class:`~pystac.STACObject` that may contain children,
    which are instances of :class:`~pystac.Catalog` or :class:`~pystac.Collection`,
    as well as :class:`~pystac.Item` s.

    Args:
        id : Identifier for the catalog. Must be unique within the STAC.
        description : Detailed multi-line description to fully explain the catalog.
            `CommonMark 0.28 syntax <https://commonmark.org/>`_ MAY be used for rich
            text representation.
        title : Optional short descriptive one-line title for the catalog.
        stac_extensions : Optional list of extensions the Catalog implements.
        href : Optional HREF for this catalog, which be set as the
            catalog's self link's HREF.
        catalog_type : Optional catalog type for this catalog. Must
            be one of the values in :class:`~pystac.CatalogType`.
    


Let's just give an ID and a description. We don't have to worry about the HREF right now;  that  will be set later.

In [15]:
catalog = pystac.Catalog(id='test-catalog', description='Tutorial catalog.')

There are no children or items in the catalog, since we haven't added anything yet.

In [16]:
print(list(catalog.get_children()))
print(list(catalog.get_items()))

[]
[]


We'll now create an Item to represent the image. Check the pydocs to see what you need to supply:

In [17]:
print(pystac.Item.__doc__)

An Item is the core granular entity in a STAC, containing the core metadata
    that enables any client to search or crawl online catalogs of spatial 'assets' -
    satellite imagery, derived data, DEM's, etc.

    Args:
        id : Provider identifier. Must be unique within the STAC.
        geometry : Defines the full footprint of the asset represented by this
            item, formatted according to
            `RFC 7946, section 3.1 (GeoJSON) <https://tools.ietf.org/html/rfc7946>`_.
        bbox :  Bounding Box of the asset represented by this item
            using either 2D or 3D geometries. The length of the array must be 2*n
            where n is the number of dimensions. Could also be None in the case of a
            null geometry.
        datetime : Datetime associated with this item. If None,
            a start_datetime and end_datetime must be supplied in the properties.
        properties : A dictionary of additional metadata for the item.
        stac_extensions : Opt

Using [rasterio](https://rasterio.readthedocs.io/en/stable/), we can pull out the bounding box of the image to use for the image metadata. If the image contained a NoData border, we would ideally pull out the footprint and save it as the geometry; in this case, we're working with a small chip that most likely has no NoData values.

In [18]:
import rasterio
from shapely.geometry import Polygon, mapping

def get_bbox_and_footprint(raster_uri):
    with rasterio.open(raster_uri) as ds:
        bounds = ds.bounds
        bbox = [bounds.left, bounds.bottom, bounds.right, bounds.top]
        footprint = Polygon([
            [bounds.left, bounds.bottom],
            [bounds.left, bounds.top],
            [bounds.right, bounds.top],
            [bounds.right, bounds.bottom]
        ])
        
        return (bbox, mapping(footprint))

In [19]:
img_path="0-croplands_2020-02-07_2021-02-01.tif"


bbox, footprint = get_bbox_and_footprint(img_path)
print(bbox)
print(footprint)

[-111.63283187377722, 35.21539644193983, -111.63094541168057, 35.216923577922834]
{'type': 'Polygon', 'coordinates': (((-111.63283187377722, 35.21539644193983), (-111.63283187377722, 35.216923577922834), (-111.63094541168057, 35.216923577922834), (-111.63094541168057, 35.21539644193983), (-111.63283187377722, 35.21539644193983)),)}


We're also using `datetime.utcnow()` to supply the required datetime property for our Item. Since this is a required property, you might often find yourself making up a time to fill in if you don't know the exact capture time.

In [20]:
from datetime import datetime

item = pystac.Item(id='local-image',
                 geometry=footprint,
                 bbox=bbox,
                 datetime=datetime.utcnow(),
                 properties={})

We haven't added it to a catalog yet, so it's parent isn't set. Once we add it to the catalog, we can see it correctly links to it's parent.

In [21]:
assert item.get_parent() is None

In [22]:
catalog.add_item(item)

In [23]:
item.get_parent()

<Catalog id=test-catalog>

`describe()` is a useful method on `Catalog` - but be careful when using it on large catalogs, as it will walk the entire tree of the STAC.

In [24]:
catalog.describe()

* <Catalog id=test-catalog>
  * <Item id=local-image>


### Adding Assets

We've created an Item, but there aren't any assets associated with it. Let's create one:

In [25]:
print(pystac.Asset.__doc__)

An object that contains a link to data associated with an Item or Collection that
    can be downloaded or streamed.

    Args:
        href : Link to the asset object. Relative and absolute links are both
            allowed.
        title : Optional displayed title for clients and users.
        description : A description of the Asset providing additional details,
            such as how it was processed or created. CommonMark 0.29 syntax MAY be used
            for rich text representation.
        media_type : Optional description of the media type. Registered Media Types
            are preferred. See :class:`~pystac.MediaType` for common media types.
        roles : Optional, Semantic roles (i.e. thumbnail, overview,
            data, metadata) of the asset.
        extra_fields : Optional, additional fields for this asset. This is used
            by extensions as a way to serialize and deserialize properties on asset
            object JSON.
    


In [26]:
item.add_asset(
    key='image', 
    asset=pystac.Asset(
        href=img_path, 
        media_type=pystac.MediaType.GEOTIFF
    )
)

At any time we can call `to_dict()` on STAC objects to see how the STAC JSON is shaping up. Notice the asset is now set:

In [27]:
import json
print(json.dumps(item.to_dict(), indent=4))

{
    "type": "Feature",
    "stac_version": "1.0.0",
    "id": "local-image",
    "properties": {
        "datetime": "2022-06-17T17:35:33.733166Z"
    },
    "geometry": {
        "type": "Polygon",
        "coordinates": [
            [
                [
                    -111.63283187377722,
                    35.21539644193983
                ],
                [
                    -111.63283187377722,
                    35.216923577922834
                ],
                [
                    -111.63094541168057,
                    35.216923577922834
                ],
                [
                    -111.63094541168057,
                    35.21539644193983
                ],
                [
                    -111.63283187377722,
                    35.21539644193983
                ]
            ]
        ]
    },
    "links": [
        {
            "rel": "root",
            "href": null,
            "type": "application/json"
        },
        {
          

Note that the link `href` properties are `null`. This is OK, as we're working with the STAC in memory. Next, we'll talk about writing the catalog out, and how to set those HREFs.

### Saving the catalog

As the JSON above indicates, there's no HREFs set on these in-memory items. PySTAC uses the `self` link on STAC objects to track where the file lives. Because we haven't set them, they evaluate to `None`:

In [28]:
print(catalog.get_self_href() is None)
print(item.get_self_href() is None)

True
True


In order to set them, we can use `normalize_hrefs`. This method will create a normalized set of HREFs for each STAC object in the catalog, according to the [best practices document](https://github.com/radiantearth/stac-spec/blob/v0.8.1/best-practices.md#catalog-layout)'s recommendations on how to lay out a catalog.

In [29]:
catalog.normalize_hrefs(os.path.join(tmp_dir.name, 'stac'))

Now that we've normalized to a root directory (the temporary directory), we see that the `self` links are set:

In [30]:
print(catalog.get_self_href())
print(item.get_self_href())

/var/folders/dr/8s58ng3j2sq3kmlvcgz913br0000gp/T/tmpm1vgch4u/stac/catalog.json
/var/folders/dr/8s58ng3j2sq3kmlvcgz913br0000gp/T/tmpm1vgch4u/stac/local-image/local-image.json


We can now call `save` on the catalog, which will recursively save all the STAC objects to their respective self HREFs.

Save requires a `CatalogType` to be set. You can review the [API docs](https://pystac.readthedocs.io/en/stable/api.html#catalogtype) on `CatalogType` to see what each type means (unfortunately `help` doesn't show docstrings for attributes).

In [31]:
catalog.save(catalog_type=pystac.CatalogType.SELF_CONTAINED)

In [32]:
!ls {tmp_dir.name}/stac/*

/var/folders/dr/8s58ng3j2sq3kmlvcgz913br0000gp/T/tmpm1vgch4u/stac/catalog.json

/var/folders/dr/8s58ng3j2sq3kmlvcgz913br0000gp/T/tmpm1vgch4u/stac/local-image:
local-image.json


In [33]:
with open(item.self_href) as f:
    print(f.read())

{
  "type": "Feature",
  "stac_version": "1.0.0",
  "id": "local-image",
  "properties": {
    "datetime": "2022-06-17T17:35:33.733166Z"
  },
  "geometry": {
    "type": "Polygon",
    "coordinates": [
      [
        [
          -111.63283187377722,
          35.21539644193983
        ],
        [
          -111.63283187377722,
          35.216923577922834
        ],
        [
          -111.63094541168057,
          35.216923577922834
        ],
        [
          -111.63094541168057,
          35.21539644193983
        ],
        [
          -111.63283187377722,
          35.21539644193983
        ]
      ]
    ]
  },
  "links": [
    {
      "rel": "root",
      "href": "../catalog.json",
      "type": "application/json"
    },
    {
      "rel": "parent",
      "href": "../catalog.json",
      "type": "application/json"
    }
  ],
  "assets": {
    "image": {
      "href": "0-croplands_2020-02-07_2021-02-01.tif",
      "type": "image/tiff; application=geotiff"
    }
  },
  "bbox"

In [34]:
with open(item.self_href) as f:
    print(f.read())

{
  "type": "Feature",
  "stac_version": "1.0.0",
  "id": "local-image",
  "properties": {
    "datetime": "2022-06-17T17:35:33.733166Z"
  },
  "geometry": {
    "type": "Polygon",
    "coordinates": [
      [
        [
          -111.63283187377722,
          35.21539644193983
        ],
        [
          -111.63283187377722,
          35.216923577922834
        ],
        [
          -111.63094541168057,
          35.216923577922834
        ],
        [
          -111.63094541168057,
          35.21539644193983
        ],
        [
          -111.63283187377722,
          35.21539644193983
        ]
      ]
    ]
  },
  "links": [
    {
      "rel": "root",
      "href": "../catalog.json",
      "type": "application/json"
    },
    {
      "rel": "parent",
      "href": "../catalog.json",
      "type": "application/json"
    }
  ],
  "assets": {
    "image": {
      "href": "0-croplands_2020-02-07_2021-02-01.tif",
      "type": "image/tiff; application=geotiff"
    }
  },
  "bbox"

As you can see, all links are saved with relative paths. That's because we used `catalog_type=CatalogType.SELF_CONTAINED`. If we save an Absolute Published catalog, we'll see absolute paths:

In [35]:
# catalog.save(catalog_type=pystac.CatalogType.ABSOLUTE_PUBLISHED)

Now the links included in the STAC item are all absolute:

In [36]:
# with open(item.get_self_href()) as f:
#     print(f.read())

Notice that the Asset HREF is absolute in both cases. We can make the Asset HREF relative to the STAC Item by using `.make_all_asset_hrefs_relative()`:

In [37]:
catalog.make_all_asset_hrefs_relative()
catalog.save(catalog_type=pystac.CatalogType.SELF_CONTAINED)

In [38]:
with open(item.get_self_href()) as f:
    print(f.read())

{
  "type": "Feature",
  "stac_version": "1.0.0",
  "id": "local-image",
  "properties": {
    "datetime": "2022-06-17T17:35:33.733166Z"
  },
  "geometry": {
    "type": "Polygon",
    "coordinates": [
      [
        [
          -111.63283187377722,
          35.21539644193983
        ],
        [
          -111.63283187377722,
          35.216923577922834
        ],
        [
          -111.63094541168057,
          35.216923577922834
        ],
        [
          -111.63094541168057,
          35.21539644193983
        ],
        [
          -111.63283187377722,
          35.21539644193983
        ]
      ]
    ]
  },
  "links": [
    {
      "rel": "root",
      "href": "../catalog.json",
      "type": "application/json"
    },
    {
      "rel": "parent",
      "href": "../catalog.json",
      "type": "application/json"
    }
  ],
  "assets": {
    "image": {
      "href": "0-croplands_2020-02-07_2021-02-01.tif",
      "type": "image/tiff; application=geotiff"
    }
  },
  "bbox"

### Creating an Item that implements the EO extension

In the code above our item only implemented the core STAC Item specification. With [extensions](https://github.com/radiantearth/stac-spec/tree/v0.9.0/extensions) we can record more information and add additional functionality to the Item. Given that we know this is a World View 3 image that has earth observation data, we can enable the [eo extension](https://github.com/radiantearth/stac-spec/tree/v0.8.1/extensions/eo) to add band information.

To add eo information to an item we'll need to specify some more data. First, let's define the bands of World View 3:

In [39]:
from pystac.extensions.eo import Band, EOExtension

# From: https://www.spaceimagingme.com/downloads/sensors/datasheets/DG_WorldView3_DS_2014.pdf

wv3_bands = [Band.create(name='Coastal', description='Coastal: 400 - 450 nm', common_name='coastal'),
             Band.create(name='Blue', description='Blue: 450 - 510 nm', common_name='blue'),
             Band.create(name='Green', description='Green: 510 - 580 nm', common_name='green'),
             Band.create(name='Yellow', description='Yellow: 585 - 625 nm', common_name='yellow'),
             Band.create(name='Red', description='Red: 630 - 690 nm', common_name='red'),
             Band.create(name='Red Edge', description='Red Edge: 705 - 745 nm', common_name='rededge'),
             Band.create(name='Near-IR1', description='Near-IR1: 770 - 895 nm', common_name='nir08'),
             Band.create(name='Near-IR2', description='Near-IR2: 860 - 1040 nm', common_name='nir09')]

Notice that we used the `.create` method create new band information.

We can now create an Item, enable the eo extension, add the band information and add it to our catalog:

In [ ]:
eo_item = pystac.Item(id='local-image-eo',
                      geometry=footprint,
                      bbox=bbox,
                      datetime=datetime.utcnow(),
                      properties={})
eo = EOExtension.ext(eo_item, add_if_missing=True)
eo.apply(bands=wv3_bands)

There are also [common metadata](https://github.com/radiantearth/stac-spec/blob/v0.9.0/item-spec/common-metadata.md) fields that we can use to capture additional information about the WorldView 3 imagery:

In [ ]:
eo_item.common_metadata.platform = "Maxar"
eo_item.common_metadata.instruments = ["WorldView3"]
eo_item.common_metadata.gsd = 0.3

In [ ]:
print(eo_item)

<Item id=local-image-eo>


We can use the eo extension to add bands to the assets we add to the item:

In [ ]:
asset = pystac.Asset(href=img_path, 
                     media_type=pystac.MediaType.GEOTIFF)
eo_item.add_asset("image", asset)
eo_on_asset = EOExtension.ext(eo_item.assets["image"])
eo_on_asset.apply(wv3_bands)

If we look at the asset's JSON representation, we can see the appropriate band indexes are set:

In [ ]:
asset.to_dict()

{'href': '/var/folders/9z/lnsvqfqj4gs2d1j1nw3vynrm0000gn/T/tmpzpx86d17/image.tif',
 'type': <MediaType.GEOTIFF: 'image/tiff; application=geotiff'>,
 'eo:bands': [{'name': 'Coastal',
   'common_name': 'coastal',
   'description': 'Coastal: 400 - 450 nm'},
  {'name': 'Blue', 'common_name': 'blue', 'description': 'Blue: 450 - 510 nm'},
  {'name': 'Green',
   'common_name': 'green',
   'description': 'Green: 510 - 580 nm'},
  {'name': 'Yellow',
   'common_name': 'yellow',
   'description': 'Yellow: 585 - 625 nm'},
  {'name': 'Red', 'common_name': 'red', 'description': 'Red: 630 - 690 nm'},
  {'name': 'Red Edge',
   'common_name': 'rededge',
   'description': 'Red Edge: 705 - 745 nm'},
  {'name': 'Near-IR1',
   'common_name': 'nir08',
   'description': 'Near-IR1: 770 - 895 nm'},
  {'name': 'Near-IR2',
   'common_name': 'nir09',
   'description': 'Near-IR2: 860 - 1040 nm'}]}

Let's clear the in-memory catalog, add the EO item, and save to a new STAC:

In [ ]:
catalog.clear_items()
list(catalog.get_items())

[]

In [ ]:
catalog.add_item(eo_item)
list(catalog.get_items())

[<Item id=local-image-eo>]

In [ ]:
catalog.normalize_and_save(root_href=os.path.join(tmp_dir.name, 'stac-eo'), 
                           catalog_type=pystac.CatalogType.SELF_CONTAINED)

Now, if we read the catalog from the filesystem, PySTAC recognizes that the item implements eo and so use it's functionality, e.g. getting the bands off the asset:

In [ ]:
catalog2 = pystac.read_file(os.path.join(tmp_dir.name, 'stac-eo', 'catalog.json'))

In [ ]:
assert isinstance(catalog2, pystac.Catalog)
list(catalog2.get_items())

[<Item id=local-image-eo>]

In [ ]:
item: pystac.Item = next(catalog2.get_all_items())

In [ ]:
assert EOExtension.has_extension(item)

In [ ]:
eo_on_asset = EOExtension.ext(item.assets["image"])
print(eo_on_asset.bands)

[<Band name=Coastal>, <Band name=Blue>, <Band name=Green>, <Band name=Yellow>, <Band name=Red>, <Band name=Red Edge>, <Band name=Near-IR1>, <Band name=Near-IR2>]


### Collections

Collections are a subtype of Catalog that have some additional properties to make them more searchable. They also can define common properties so that items in the collection don't have to duplicate common data for each item. Let's create a collection to hold common properties between two images from the Spacenet 5 challenge.

First we'll get another image, and it's bbox and footprint:

In [ ]:
url2 = ('https://spacenet-dataset.s3.amazonaws.com/'
       'spacenet/SN5_roads/train/AOI_7_Moscow/MS/'
       'SN5_roads_train_AOI_7_Moscow_MS_chip997.tif')
img_path2 = os.path.join(tmp_dir.name, 'image.tif')
urllib.request.urlretrieve(url2, img_path2)

('/var/folders/9z/lnsvqfqj4gs2d1j1nw3vynrm0000gn/T/tmpzpx86d17/image.tif',
 <http.client.HTTPMessage at 0x114dba130>)

In [ ]:
bbox2, footprint2 = get_bbox_and_footprint(img_path2)

We can take a look at the pydocs for Collection to see what information we need to supply in order to satisfy the spec.

In [ ]:
print(pystac.Collection.__doc__)

A Collection extends the Catalog spec with additional metadata that helps
    enable discovery.

    Args:
        id : Identifier for the collection. Must be unique within the STAC.
        description : Detailed multi-line description to fully explain the
            collection. `CommonMark 0.28 syntax <https://commonmark.org/>`_ MAY
            be used for rich text representation.
        extent : Spatial and temporal extents that describe the bounds of
            all items contained within this Collection.
        title : Optional short descriptive one-line title for the
            collection.
        stac_extensions : Optional list of extensions the Collection
            implements.
        href : Optional HREF for this collection, which be set as the
            collection's self link's HREF.
        catalog_type : Optional catalog type for this catalog. Must
            be one of the values in :class`~pystac.CatalogType`.
        license :  Collection's license(s) as a
     

Beyond what a Catalog reqiures, a Collection requires a license, and an `Extent` that describes the range of space and time that the items it hold occupy.

In [ ]:
print(pystac.Extent.__doc__)

Describes the spatiotemporal extents of a Collection.

    Args:
        spatial : Potential spatial extent covered by the collection.
        temporal : Potential temporal extent covered by the collection.
        extra_fields : Dictionary containing additional top-level fields defined on the
            Extent object.
    


An Extent is comprised of a SpatialExtent and a TemporalExtent. These hold one or more bounding boxes and time intervals, respectively, that completely  cover the items contained in the collections.

Let's start with creating two new items - these will be core Items. We can set these items to implement the `eo` extension by specifying them in the `stac_extensions`.

In [ ]:
collection_item = pystac.Item(id='local-image-col-1',
                               geometry=footprint,
                               bbox=bbox,
                               datetime=datetime.utcnow(),
                               properties={})

collection_item.common_metadata.gsd = 0.3
collection_item.common_metadata.platform = 'Maxar'
collection_item.common_metadata.instruments = ['WorldView3']

asset = pystac.Asset(href=img_path, 
                      media_type=pystac.MediaType.GEOTIFF)
collection_item.add_asset("image", asset)
eo = EOExtension.ext(collection_item.assets["image"], add_if_missing=True)
eo.apply(wv3_bands)

collection_item2 = pystac.Item(id='local-image-col-2',
                               geometry=footprint2,
                               bbox=bbox2,
                               datetime=datetime.utcnow(),
                               properties={})

collection_item2.common_metadata.gsd = 0.3
collection_item2.common_metadata.platform = 'Maxar'
collection_item2.common_metadata.instruments = ['WorldView3']

asset2 = pystac.Asset(href=img_path,
                     media_type=pystac.MediaType.GEOTIFF)
collection_item2.add_asset("image", asset2)
eo = EOExtension.ext(collection_item2.assets["image"], add_if_missing=True)
eo.apply([
    band for band in wv3_bands if band.name in ["Red", "Green", "Blue"]
])


We can use our two items' metadata to find out what the proper bounds are:

In [ ]:
from shapely.geometry import shape

unioned_footprint = shape(footprint).union(shape(footprint2))
collection_bbox = list(unioned_footprint.bounds)
spatial_extent = pystac.SpatialExtent(bboxes=[collection_bbox])

In [ ]:
collection_interval = sorted([collection_item.datetime, collection_item2.datetime])
temporal_extent = pystac.TemporalExtent(intervals=[collection_interval])

In [ ]:
collection_extent = pystac.Extent(spatial=spatial_extent, temporal=temporal_extent)

In [ ]:
collection = pystac.Collection(id='wv3-images',
                               description='Spacenet 5 images over Moscow',
                               extent=collection_extent,
                               license='CC-BY-SA-4.0')

Now if we add our items to our Collection, and our Collection to our Catalog, we get the following STAC that can be saved:

In [ ]:
collection.add_items([collection_item, collection_item2])

In [ ]:
catalog.clear_items()
catalog.clear_children()
catalog.add_child(collection)

In [ ]:
catalog.describe()

* <Catalog id=test-catalog>
    * <Collection id=wv3-images>
      * <Item id=local-image-col-1>
      * <Item id=local-image-col-2>


In [ ]:
catalog.normalize_and_save(root_href=os.path.join(tmp_dir.name, 'stac-collection'), 
                           catalog_type=pystac.CatalogType.SELF_CONTAINED)

### Cleanup

Don't forget to clean up the temporary directory!

In [ ]:
tmp_dir.cleanup()

## Creating a STAC of imagery from Spacenet 5 data

Now, let's take what we've learned and create a Catalog with more data in it.


### Allowing PySTAC to read from AWS S3

PySTAC aims to be virtually zero-dependency (notwithstanding the why-isn't-this-in-stdlib datetime-util), so it doesn't have the ability to read from or write to anything but the local file system. However, we can hook into PySTAC's IO in the following way. Learn more about how to customize I/O in STAC from the [documentation](https://pystac.readthedocs.io/en/stable/concepts.html#i-o-in-pystac):

In [ ]:
from typing import Union, Any
from urllib.parse import urlparse

import boto3
from pystac import Link
from pystac.stac_io import DefaultStacIO


class CustomStacIO(DefaultStacIO):
   def __init__(self):
      self.s3 = boto3.resource("s3")

   def read_text(
      self, source: Union[str, Link], *args: Any, **kwargs: Any
   ) -> str:
      parsed = urlparse(uri)
      if parsed.scheme == "s3":
         bucket = parsed.netloc
         key = parsed.path[1:]

         obj = self.s3.Object(bucket, key)
         return obj.get()["Body"].read().decode("utf-8")
      else:
         return super().read_text(source, *args, **kwargs)

   def write_text(
      self, dest: Union[str, Link], txt: str, *args: Any, **kwargs: Any
   ) -> None:
      parsed = urlparse(uri)
      if parsed.scheme == "s3":
         bucket = parsed.netloc
         key = parsed.path[1:]
         self.s3.Object(bucket, key).put(Body=txt, ContentEncoding="utf-8")
      else:
         super().write_text(dest, txt, *args, **kwargs)


We'll need a utility to list keys for reading the lists of files from S3:

In [ ]:
# From https://alexwlchan.net/2017/07/listing-s3-keys/
from botocore import UNSIGNED
from botocore.config import Config

def get_s3_keys(bucket, prefix):
    """Generate all the keys in an S3 bucket."""
    s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))
    kwargs = {'Bucket': bucket, 'Prefix': prefix}
    while True:
        resp = s3.list_objects_v2(**kwargs)
        for obj in resp['Contents']:
            yield obj['Key']

        try:
            kwargs['ContinuationToken'] = resp['NextContinuationToken']
        except KeyError:
            break

Let's make a STAC of imagery over Moscow as part of the Spacenet 5 challenge. As a first step, we can list out the imagery and extract IDs from each of the chips.

In [ ]:
moscow_training_chip_uris = list(get_s3_keys(bucket='spacenet-dataset', 
                                             prefix='spacenet/SN5_roads/train/AOI_7_Moscow/PS-MS/'))

In [ ]:
import re

chip_id_to_data = {}

def get_chip_id(uri):
    return re.search(r'.*\_chip(\d+)\.', uri).group(1)

for uri in moscow_training_chip_uris:
    chip_id = get_chip_id(uri)
    chip_id_to_data[chip_id] = { 'img': 's3://spacenet-dataset/{}'.format(uri) }

For this tutorial, we'll only take a subset of the data.

In [ ]:
chip_id_to_data = dict(list(chip_id_to_data.items())[:10])

In [ ]:
chip_id_to_data

{'0': {'img': 's3://spacenet-dataset/spacenet/SN5_roads/train/AOI_7_Moscow/PS-MS/SN5_roads_train_AOI_7_Moscow_PS-MS_chip0.tif'},
 '1': {'img': 's3://spacenet-dataset/spacenet/SN5_roads/train/AOI_7_Moscow/PS-MS/SN5_roads_train_AOI_7_Moscow_PS-MS_chip1.tif'},
 '10': {'img': 's3://spacenet-dataset/spacenet/SN5_roads/train/AOI_7_Moscow/PS-MS/SN5_roads_train_AOI_7_Moscow_PS-MS_chip10.tif'},
 '100': {'img': 's3://spacenet-dataset/spacenet/SN5_roads/train/AOI_7_Moscow/PS-MS/SN5_roads_train_AOI_7_Moscow_PS-MS_chip100.tif'},
 '1000': {'img': 's3://spacenet-dataset/spacenet/SN5_roads/train/AOI_7_Moscow/PS-MS/SN5_roads_train_AOI_7_Moscow_PS-MS_chip1000.tif'},
 '1001': {'img': 's3://spacenet-dataset/spacenet/SN5_roads/train/AOI_7_Moscow/PS-MS/SN5_roads_train_AOI_7_Moscow_PS-MS_chip1001.tif'},
 '1002': {'img': 's3://spacenet-dataset/spacenet/SN5_roads/train/AOI_7_Moscow/PS-MS/SN5_roads_train_AOI_7_Moscow_PS-MS_chip1002.tif'},
 '1003': {'img': 's3://spacenet-dataset/spacenet/SN5_roads/train/AOI_7_Mo

Let's turn each of those chips into a STAC Item that represents the image.

In [ ]:
chip_id_to_items = {}

We'll create core `Item`s for our imagery, but mark them with the `eo` extension as we did above, and store the `eo` data in a `Collection`.

Note that the image CRS is in WGS:84 (Lat/Lng). If it wasn't, we'd have to reproject the footprint to WGS:84 in order to be compliant with the spec (which can easily be done with [pyproj](https://github.com/pyproj4/pyproj)).

Here we're taking advantage of `rasterio`'s ability to read S3 URIs, which only grabs the GeoTIFF metadata and does not pull the whole file down.

In [ ]:
import os
os.environ["AWS_NO_SIGN_REQUEST"] = "true"

for chip_id in chip_id_to_data:
    img_uri = chip_id_to_data[chip_id]['img']
    print('Processing {}'.format(img_uri))
    bbox, footprint = get_bbox_and_footprint(img_uri)

    item = pystac.Item(id='img_{}'.format(chip_id), 
                       geometry=footprint,
                       bbox=bbox,
                       datetime=datetime.utcnow(),
                       properties={})
    
    item.common_metadata.gsd = 0.3
    item.common_metadata.platform = 'Maxar'
    item.common_metadata.instruments = ['WorldView3']
    
    eo = EOExtension.ext(item, add_if_missing=True)
    eo.bands = wv3_bands
    asset = pystac.Asset(href=img_uri,
                         media_type=pystac.MediaType.COG)
    item.add_asset(key='ps-ms', asset=asset)
    eo = EOExtension.ext(item.assets["ps-ms"])
    eo.bands = wv3_bands
    chip_id_to_items[chip_id] = item

Processing s3://spacenet-dataset/spacenet/SN5_roads/train/AOI_7_Moscow/PS-MS/SN5_roads_train_AOI_7_Moscow_PS-MS_chip0.tif
Processing s3://spacenet-dataset/spacenet/SN5_roads/train/AOI_7_Moscow/PS-MS/SN5_roads_train_AOI_7_Moscow_PS-MS_chip1.tif
Processing s3://spacenet-dataset/spacenet/SN5_roads/train/AOI_7_Moscow/PS-MS/SN5_roads_train_AOI_7_Moscow_PS-MS_chip10.tif
Processing s3://spacenet-dataset/spacenet/SN5_roads/train/AOI_7_Moscow/PS-MS/SN5_roads_train_AOI_7_Moscow_PS-MS_chip100.tif
Processing s3://spacenet-dataset/spacenet/SN5_roads/train/AOI_7_Moscow/PS-MS/SN5_roads_train_AOI_7_Moscow_PS-MS_chip1000.tif
Processing s3://spacenet-dataset/spacenet/SN5_roads/train/AOI_7_Moscow/PS-MS/SN5_roads_train_AOI_7_Moscow_PS-MS_chip1001.tif
Processing s3://spacenet-dataset/spacenet/SN5_roads/train/AOI_7_Moscow/PS-MS/SN5_roads_train_AOI_7_Moscow_PS-MS_chip1002.tif
Processing s3://spacenet-dataset/spacenet/SN5_roads/train/AOI_7_Moscow/PS-MS/SN5_roads_train_AOI_7_Moscow_PS-MS_chip1003.tif
Processin

### Creating the Collection

All of these images are over Moscow. In Spacenet 5, we have a couple cities that have imagery; a good way to separate these collections of imagery. We can store all of the common `eo` metadata in the collection.

In [ ]:
from shapely.geometry import (shape, MultiPolygon)

footprints = list(map(lambda i: shape(i.geometry).envelope, 
                      chip_id_to_items.values()))
collection_bbox = MultiPolygon(footprints).bounds
spatial_extent = pystac.SpatialExtent(bboxes=[collection_bbox])

In [ ]:
datetimes = sorted(list(map(lambda i: i.datetime,
                            chip_id_to_items.values())))
temporal_extent = pystac.TemporalExtent(intervals=[[datetimes[0], datetimes[-1]]])

In [ ]:
collection_extent = pystac.Extent(spatial=spatial_extent, temporal=temporal_extent)

In [ ]:
collection = pystac.Collection(id='wv3-images',
                               description='Spacenet 5 images over Moscow',
                               extent=collection_extent,
                               license='CC-BY-SA-4.0')

In [ ]:
collection.add_items(chip_id_to_items.values())

In [ ]:
collection.describe()

* <Collection id=wv3-images>
  * <Item id=img_0>
  * <Item id=img_1>
  * <Item id=img_10>
  * <Item id=img_100>
  * <Item id=img_1000>
  * <Item id=img_1001>
  * <Item id=img_1002>
  * <Item id=img_1003>
  * <Item id=img_1004>
  * <Item id=img_1005>


Now, we can create a Catalog and add the collection.

In [ ]:
catalog = pystac.Catalog(id='spacenet5', description='Spacenet 5 Data (Test)')
catalog.add_child(collection)

In [ ]:
catalog.describe()

* <Catalog id=spacenet5>
    * <Collection id=wv3-images>
      * <Item id=img_0>
      * <Item id=img_1>
      * <Item id=img_10>
      * <Item id=img_100>
      * <Item id=img_1000>
      * <Item id=img_1001>
      * <Item id=img_1002>
      * <Item id=img_1003>
      * <Item id=img_1004>
      * <Item id=img_1005>


## Adding items with the label extension to the Spacenet 5 catalog

We can use the [label extension](https://github.com/radiantearth/stac-spec/tree/v0.8.1/extensions/label) of the STAC spec to represent the training data in our STAC. For this, we need to grab the URIs of the GeoJSON of roads:

In [ ]:
moscow_training_geojson_uris = list(get_s3_keys(bucket='spacenet-dataset',
                                               prefix='spacenet/SN5_roads/train/AOI_7_Moscow/geojson_roads_speed/'))

In [ ]:
for uri in moscow_training_geojson_uris:
    chip_id = get_chip_id(uri)
    if chip_id in chip_id_to_data:
        chip_id_to_data[chip_id]['label'] = 's3://spacenet-dataset/{}'.format(uri)

We'll add the items to their own subcatalog; since they don't inherit the Collection's `eo` properties, they shouldn't go in the Collection.

In [ ]:
label_catalog = pystac.Catalog(id='spacenet-data-labels', description='Labels for Spacenet 5')
catalog.add_child(label_catalog)

To see the required fields for the label extension we can check the pydocs on the `apply` method of the extension:

In [ ]:
from pystac.extensions.label import LabelExtension
print(LabelExtension.apply.__doc__)

Applies label extension properties to the extended Item.

        Args:
            label_description : A description of the label, how it was created,
                and what it is recommended for
            label_type : An Enum of either vector label type or raster label type. Use
                one of :class:`~pystac.LabelType`.
            label_properties : These are the names of the property field(s) in each
                Feature of the label asset's FeatureCollection that contains the classes
                (keywords from label:classes if the property defines classes).
                If labels are rasters, this should be None.
            label_classes : Optional, but required if using categorical data.
                A list of :class:`LabelClasses` instances defining the list of possible
                class names for each label:properties. (e.g., tree, building, car,
                hippo)
            label_tasks : Recommended to be a subset of 'regression', 'classifi

This loop creates our label items and associates each to the appropriate source image Item.

In [ ]:
from pystac.extensions.label import LabelType

for chip_id in chip_id_to_data:
    img_item = collection.get_item('img_{}'.format(chip_id))
    assert img_item
    label_uri = chip_id_to_data[chip_id]['label']
    
    label_item = pystac.Item(id='label_{}'.format(chip_id),
                             geometry=img_item.geometry,
                             bbox=img_item.bbox,
                             datetime=datetime.utcnow(),
                             properties={})
    label = LabelExtension.ext(label_item, add_if_missing=True)
    label.apply(label_description="SpaceNet 5 Road labels",
                               label_type=LabelType.VECTOR,
                               label_tasks=['segmentation', 'regression'])
    label.add_source(img_item)
    label.add_geojson_labels(label_uri)
    
    label_catalog.add_item(label_item)

Now we have a STAC of training data!

In [ ]:
catalog.describe()

* <Catalog id=spacenet5>
    * <Collection id=wv3-images>
      * <Item id=img_0>
      * <Item id=img_1>
      * <Item id=img_10>
      * <Item id=img_100>
      * <Item id=img_1000>
      * <Item id=img_1001>
      * <Item id=img_1002>
      * <Item id=img_1003>
      * <Item id=img_1004>
      * <Item id=img_1005>
    * <Catalog id=spacenet-data-labels>
      * <Item id=label_0>
      * <Item id=label_1>
      * <Item id=label_10>
      * <Item id=label_100>
      * <Item id=label_1000>
      * <Item id=label_1001>
      * <Item id=label_1002>
      * <Item id=label_1003>
      * <Item id=label_1004>
      * <Item id=label_1005>


In [ ]:
label_item = catalog.get_child('spacenet-data-labels').get_item('label_1')
label_item.to_dict()

{'type': 'Feature',
 'stac_version': '1.0.0',
 'id': 'label_1',
 'properties': {'label:description': 'SpaceNet 5 Road labels',
  'label:type': <LabelType.VECTOR: 'vector'>,
  'label:properties': None,
  'label:tasks': ['segmentation', 'regression'],
  'datetime': '2022-03-29T12:58:05.404487Z'},
 'geometry': {'type': 'Polygon',
  'coordinates': (((37.68191035616281, 55.73478210707574),
    (37.68191035616281, 55.73882710285011),
    (37.68595535193718, 55.73882710285011),
    (37.68595535193718, 55.73478210707574),
    (37.68191035616281, 55.73478210707574)),)},
 'links': [{'rel': 'source',
   'href': None,
   'type': <MediaType.JSON: 'application/json'>},
  {'rel': <RelType.ROOT: 'root'>,
   'href': None,
   'type': <MediaType.JSON: 'application/json'>},
  {'rel': <RelType.PARENT: 'parent'>,
   'href': None,
   'type': <MediaType.JSON: 'application/json'>}],
 'assets': {'labels': {'href': 's3://spacenet-dataset/spacenet/SN5_roads/train/AOI_7_Moscow/geojson_roads_speed/SN5_roads_train_A